In [21]:
import tensorflow as tf

In [4]:
x = [-.5,.5,.5,.5,.5]
y = tf.nn.relu(x)
print(y)


tf.Tensor([0.  0.5 0.5 0.5 0.5], shape=(5,), dtype=float32)


In [5]:
print(tf.reduce_mean(y))

tf.Tensor(0.4, shape=(), dtype=float32)


In [22]:
# This function to generate evidence is used for the first example
def relu_evidence(logits):
    return tf.nn.relu(logits)

# This one usually works better and used for the second and third examples
# For general settings and different datasets, you may try this one first
def exp_evidence(logits): 
    return tf.exp(tf.clip_by_value(logits,-10,10))

# This one is another alternative and 
# usually behaves better than the relu_evidence 
def softplus_evidence(logits):
    return tf.nn.softplus(logits)
K=10
def KL(alpha):
    # print("K:",K)
    beta=tf.constant(np.ones((1,K)),dtype=tf.float32)
    S_alpha = tf.cast(tf.reduce_sum(alpha,axis=1,keepdims=True),tf.float32)
    
    print("lgama",tf.compat.v1.lgamma(S_alpha) )
    S_beta = tf.reduce_sum(beta,axis=1,keepdims=True)
    lnB = tf.compat.v1.lgamma(S_alpha) - tf.reduce_sum(tf.compat.v1.lgamma(alpha),axis=1,keepdims=True)
    lnB_uni = tf.reduce_sum(tf.compat.v1.lgamma(beta),axis=1,keepdims=True) - tf.compat.v1.lgamma(S_beta)
    
    dg0 = tf.compat.v1.digamma(S_alpha)
    dg1 = tf.compat.v1.digamma(alpha)
    
    kl = tf.reduce_sum((alpha - beta)*(dg1-dg0),axis=1,keepdims=True) + lnB + lnB_uni
    # print("kl", kl)
    return kl
def mse_loss_global(labels, outputs): 
#         labels = tf.one_hot(tf.cast(labels, tf.int32), 10)
# #         print("onehot",labels)
#         labels = tf.cast(labels, dtype=tf.float32)
#         try:
#             labels= tf.squeeze(labels,[1])
#         except:
#                 print("loss labels can't be squeezed")
        print("global Loss")
        evidence = softplus_evidence(outputs)
        print("evid_relu",evidence)
        alpha = evidence + 1
        alpha = tf.cast(alpha, tf.float32)
        print("alpha",alpha)
        S = tf.reduce_sum(alpha, axis=1, keepdims=True) 
        E = alpha - 1
        m = alpha / S
#         print("m",m)
#         print("global", labels)
        A = tf.reduce_sum((labels-m)**2, axis=1, keepdims=True) 
        B = tf.reduce_sum(alpha*(S-alpha)/(S*S*(S+1)), axis=1, keepdims=True) 

        annealing_coef = tf.minimum(1.0,tf.cast(1,tf.float32))
#         annealing_coef = 1
        alp = E*(1-labels) + 1 
        print("alp", alp)
        C =  annealing_coef * KL(alp)
#         print(alpha)
        # C = keras_kl(labels, alpha)
        return (A + B) + C


In [30]:
print(tf.nn.softplus(0.408539593))

tf.Tensor(0.9181366, shape=(), dtype=float32)


In [35]:
import numpy as np
labels =  np.array([[0., 0., 0., 0., 0., 0., 0. ,0. ,1., 0.]])
# outputs = np.array([[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]])
outputs = np.array([[3,3,4,0,1,3,3,3,3,3]])


# # outputs = np.array([[-6,-3,3,-4,-6,-3,-3,-3,-3,-3]])
# print("mse", mse_loss_global(labels,outputs))
# print("evidence",tf.reduce_sum(relu_evidence(outputs)))

# print("keras_mse",tf.keras.losses.mean_squared_error(labels,relu_evidence(outputs)))
# print("keras_mae",tf.keras.losses.mean_absolute_error(labels,(outputs)))
# print("keras_crossE",tf.keras.losses.categorical_crossentropy(labels,tf.cast(outputs,tf.float32),True))
# print("softmax", tf.nn.softmax(tf.cast(outputs,tf.float32)),"label:",tf.argmax(outputs,1))


outputs = np.array([[-0.867575228, 0.408539593, -5.3238, -4.99858856, -4.85228109, -6.52761173, -6.87370205, -6.45510149, 0.890240133 ,-2.260957]])
# outputs = np.array([[1.2,0,1.1,1,0,0,0,0,0,0]])
outputs = tf.cast(outputs,tf.float32)
print("mse", mse_loss_global(labels,outputs))
print("KL",KL(softplus_evidence(outputs))+1)
print("KL2",KL((outputs))*.1)
print("keras KL",tf.keras.losses.kl_divergence(labels,outputs))


print("evidence",tf.reduce_sum(softplus_evidence(outputs)))

print("keras_mse",tf.keras.losses.mean_squared_error(labels,softplus_evidence(outputs)+1))
# print("keras_mae",tf.keras.losses.mean_absolute_error(labels,(outputs)))
print("keras_crossE",tf.keras.losses.categorical_crossentropy(labels,tf.cast(outputs,tf.float32),True))
print("softmax", tf.nn.softmax(tf.cast(outputs,tf.float32)), "label:",tf.argmax(outputs,1))


global Loss
evid_relu tf.Tensor(
[[3.5063478e-01 9.1813660e-01 4.8623555e-03 6.7248014e-03 7.7801961e-03
  1.4614260e-03 1.0341047e-03 1.5712447e-03 1.2342249e+00 9.9166982e-02]], shape=(1, 10), dtype=float32)
alpha tf.Tensor(
[[1.3506348 1.9181366 1.0048623 1.0067248 1.0077802 1.0014614 1.0010341
  1.0015713 2.2342248 1.099167 ]], shape=(1, 10), dtype=float32)
alp tf.Tensor(
[[1.3506348 1.9181366 1.0048623 1.0067248 1.0077802 1.0014614 1.0010341
  1.0015713 1.        1.099167 ]], shape=(1, 10), dtype=float32)
lgama tf.Tensor([[16.032024]], shape=(1, 1), dtype=float32)
mse tf.Tensor([[1.1837213]], shape=(1, 1), dtype=float32)
lgama tf.Tensor([[0.37678945]], shape=(1, 1), dtype=float32)
KL tf.Tensor([[2735.4424]], shape=(1, 1), dtype=float32)
lgama tf.Tensor([[-96.82233]], shape=(1, 1), dtype=float32)
KL2 tf.Tensor([[402.2756]], shape=(1, 1), dtype=float32)
keras KL tf.Tensor([0.11626254], shape=(1,), dtype=float32)
evidence tf.Tensor(2.6255975, shape=(), dtype=float32)
keras_mse tf.Ten

In [8]:
outputs = np.array([[-6,-3,3,-4,-6,-3,-3,-3,-3,-3]])
outputs = np.array([[0,0,0,0,0,0,0,0,0,0]])
outputs = tf.cast(outputs,tf.float32)
print("evidence",tf.reduce_sum(softplus_evidence(outputs)))


evidence tf.Tensor(6.931472, shape=(), dtype=float32)


In [19]:
import numpy as np
x = np.array([.01,1,6])
y = np.array([0,0,1])
print(tf.keras.losses.mean_squared_error(y,x))

tf.Tensor(8.6667, shape=(), dtype=float64)


In [20]:
out = y - x
print(out)
out = tf.reduce_mean(out*out)
out

[-0.01 -1.   -5.  ]


<tf.Tensor: shape=(), dtype=float64, numpy=8.6667>